In [1]:
a1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [49]:
import math

def init_computer(code, inputs):
    return {
        'mem': code.copy(),
        'inst': 0,
        'inputs': inputs.copy(),
        'outputs': [],
        'halt': False,
        'needs_input': False
    }

def run(computer):
    code_size = len(computer['mem'])
    b = computer['mem']
    i = computer['inst']
    outputs = []
    op_info = {1:3, 2:3, 3:1, 4:1, 5:2, 6:2, 7:3, 8:3, 99:0}
    while(i < code_size):
        opcode = b[i] % 100
        if(not(opcode in op_info)):
            print("error unknown opcode %i" % (opcode))
            computer['needs_input'] = False
            break
        a0 = -1
        a1 = -1
        a2 = -1
        jump = False
        if(op_info[opcode] > 0):
            p_mode = (math.floor(b[i] / 100) % 10)
            if( p_mode == 0 ):
                a0 = b[i + 1]
                #print("arg0 in ptr mode")
            elif( p_mode == 1 ):
                #print("arg0 in imm mode")
                a0 = i + 1
        if(op_info[opcode] > 1):
            p_mode = (math.floor(b[i] / 1000) % 10)
            if( p_mode == 0 ):
                #print("arg1 in ptr mode")
                a1 = b[i + 2]
            elif( p_mode == 1 ):
                a1 = i + 2
                #print("arg1 in imm mode: %i" % a1)
        if(op_info[opcode] > 2):
            p_mode = (math.floor(b[i] / 10000) % 10)
            if( p_mode == 0 ):
                #print("arg2 in ptr mode")
                a2 = b[i + 3]
            elif( p_mode == 1 ):
                #print("arg2 in imm mode")
                a2 = i + 3
        if(opcode == 1):
            #print("add %i + %i" % (b[a0], b[a1]))
            #print("before pos(%i) = %i + %i" % (i + 3, b[b[i + 1]], b[b[i + 2]]))
            b[a2] = b[a0] + b[a1]
        elif(opcode == 2):
            #print("mult %i x %i" % (b[a0], b[a1]))
            b[a2] = b[a0] * b[a1]
        elif(opcode == 3):
            if(len(computer['inputs']) == 0):
                #print("waiting on input")
                computer['needs_input'] = True
                break
            b[a0] = computer['inputs'][0]
            computer['inputs'] = computer['inputs'][1:]
        elif(opcode == 4):
            #print("out: %i" % (b[a0]))
            outputs.append(b[a0])
        elif(opcode == 5):
            if(b[a0] != 0):
                jump = True
                i = b[a1]
        elif(opcode == 6):
            if(b[a0] == 0):
                jump = True
                i = b[a1]
        elif(opcode == 7):
            b[a2] = 1 if(b[a0] < b[a1]) else 0
        elif(opcode == 8):
            b[a2] = 1 if(b[a0] == b[a1]) else 0
        elif(opcode == 99):
            #print('halting due to opcode 99')
            computer['halt'] = True
            computer['needs_input'] = False
            break
        if(not(jump)):
            i = i + op_info[opcode] + 1
        if(i >= code_size):
            print('exiting b/c end of code reached')
            computer['needs_input'] = False
    computer['outputs'] = outputs
    computer['mem'] = b
    computer['inst'] = i
    
    return computer

def run_amps(code, setting):
    compA = init_computer(code, [setting[0], 0])
    compB = init_computer(code, [setting[1]])
    compC = init_computer(code, [setting[2]])
    compD = init_computer(code, [setting[3]])
    compE = init_computer(code, [setting[4]])

    while(1):
        compA = run(compA)
        a_out = compA['outputs']
        compA['outputs'] = []
        compB['inputs'] = compB['inputs'] + a_out

        compB = run(compB)
        b_out = compB['outputs']
        compB['outputs'] = []
        compC['inputs'] = compC['inputs'] + b_out

        compC = run(compC)
        c_out = compC['outputs']
        compC['outputs'] = []
        compD['inputs'] = compD['inputs'] + c_out

        compD = run(compD)
        d_out = compD['outputs']
        compD['outputs'] = []
        compE['inputs'] = compE['inputs'] + d_out
        compE = run(compE)
        
        if(compE['halt']):
            break
        
        e_out = compE['outputs']
        
        if(len(e_out) == 0):
            if((compA['needs_input'] or compA['halt']) and 
               (compB['needs_input'] or compB['halt']) and 
               (compC['needs_input'] or compC['halt']) and 
               (compD['needs_input'] or compD['halt']) and 
               (compE['needs_input'] or compE['halt'])):
                return "nope: lacking input and/or halted"
        
        compE['outputs'] = []
        compA['inputs'] = compA['inputs'] + e_out
    
    return compE['outputs'][0]


setting = [4,3,2,1,0]
output = run_amps(a1, setting)
print("test 1: %i" % output)

a2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
setting = [0,1,2,3,4]
output = run_amps(a2, setting)
print("test 2: %i" % output)


test 1: 43210
test 2: 54321


In [50]:
def perm(l):
    if(len(l) < 2):
        return [l]
    
    p = []
    for i in range(len(l)):
        first = l[i]
        rest = l[:i] + l[i+1:]
        for partial_perm in perm(rest):
            p.append([first] + partial_perm)
    return p

def find_optimal(code, options):
    settings = perm(options)
    highest = []
    highest_power = 0
    for setting in settings:
        power_output = run_amps(code, setting)
        if(power_output > highest_power):
            highest_power = power_output
            highest = setting
    return [highest, highest_power]

optimal = find_optimal([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [0,1,2,3,4])
print(optimal)

optimal = find_optimal([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0],[0,1,2,3,4])
print(optimal)

part1_code = [3,8,1001,8,10,8,105,1,0,0,21,46,55,68,89,110,191,272,353,434,99999,3,9,1002,9,3,9,1001,9,3,9,102,4,9,9,101,4,9,9,1002,9,5,9,4,9,99,3,9,102,3,9,9,4,9,99,3,9,1001,9,5,9,102,4,9,9,4,9,99,3,9,1001,9,5,9,1002,9,2,9,1001,9,5,9,1002,9,3,9,4,9,99,3,9,101,3,9,9,102,3,9,9,101,3,9,9,1002,9,4,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99]
print("part1:")
print(find_optimal(part1_code, [0,1,2,3,4]))

[[4, 3, 2, 1, 0], 43210]
[[1, 0, 4, 3, 2], 65210]
part1:
[[3, 2, 4, 0, 1], 440880]


In [51]:
part2_options = [5,6,7,8,9]
print(find_optimal([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], part2_options))

[[9, 8, 7, 6, 5], 139629729]


In [52]:
print(find_optimal([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10], part2_options))

[[9, 7, 8, 5, 6], 18216]


In [53]:
print(find_optimal(part1_code, part2_options))

[[5, 7, 9, 6, 8], 3745599]
